In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [2]:
import chess
from chess import pgn

In [3]:
from chess import pgn

def games_in(file):
    """
    Returns the list of games from a pgn file
    """
    games = open(file)
    game = pgn.read_game(games)
    games_list = []
    while game is not None:
        games_list.append(game)
        game = pgn.read_game(games)
    return games_list

In [4]:
from os import listdir
from os.path import isfile, join

def files_in(path):
    return [join(path,f) for f in listdir(path) if isfile(join(path, f))]

In [5]:
PIECES = chess.PIECE_TYPES

In [6]:
curpath = "pgn_games/Lichess Elite Database"

In [7]:
files = files_in(curpath)[: 20]

In [8]:
gameslist = []
for file in files:
    gameslist.extend(games_in(file))

In [12]:
columns = ['wMate', 'bMate', 'draw', 'turn', 
           'wP','wN','wB','wR','wQ','wK','bP','bN','bB','bR','bQ','bK',
           'hwP','hwN','hwB','hwR','hwQ','hwK','hbP','hbN','hbB','hbR','hbQ','hbK',
          'EVAL']

def features(board):
    """
    Returns a list of numerical values for features of a board. Subject to change.
    [ischeckmate[W/B], isDraw, turn, material[PNBRKQ][W/B] ,hangingMaterial[W/b]]
    """
    featuresList = []
    featuresList.extend(colorList(isCheckmate, board))
    featuresList.extend([isDraw(board)])
    featuresList.extend([currentTurn(board)])
    featuresList.extend(pieceColorList(numPieceOfColor, board))
    featuresList.extend(pieceColorList(numHangingPieceOfColor, board))
    return featuresList
    
def pieceColorList(fn, board):
    """
    Returns a list containing FN called on every piece type and color combo
    """
    a = [fn(board, piece, chess.WHITE) for piece in PIECES]
    a.extend([fn(board, piece, chess.BLACK)for piece in PIECES])
    return a
    
def colorList(fn, board):
    """
    Returns a list containing FN called on both color
    """
    a = [fn(board, chess.WHITE)]
    a.extend([fn(board, chess.BLACK)])
    return a
    
def isCheckmate(board, color):
    """
    Returns 1 if the current color is checkmating the other
    """
    if board.is_checkmate() and board.turn == (not color):
        return 1
    return 0

def isDraw(board):
    """
    Returns 1 iff the current position is a draw
    """
    if board.result() == '1/2-1/2': #might have to input some * in result
        return 1
    return 0

def currentTurn(board):
    if board.turn == chess.WHITE:
        return 1
    return 0

def numPieceOfColor(board, piece, color):
    """
    Returns the number of pieces of specific color
    """
    return len(board.pieces(piece, color))

def numHangingPieceOfColor(board, piece, color): 
    #Update function to still count pieces as hanging if num attacker pawns > defender pawns, repeat for each piece...
    """
    Returns the number of hanging pieces of a specific color 
    """
    our_pieces = board.pieces(piece, color)
    opp_color = not color
    hanging_pieces = 0
    for sq in our_pieces:
        unpinned_attackers_on_sq = sum([not board.is_pinned(not color, opp_att) for opp_att in board.attackers(opp_color, sq)])
        unpinned_defenders_on_sq = sum([not board.is_pinned(color, opp_att) for opp_att in board.attackers(color, sq)]) 
        if unpinned_attackers_on_sq > unpinned_defenders_on_sq:
            hanging_pieces += 1
    return hanging_pieces
        

In [13]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
testBoard.push(chess.Move.from_uci('d1h5'))
testBoard.push(chess.Move.from_uci('g8f6'))
testBoard.push(chess.Move.from_uci('f1c4'))
testBoard.push(chess.Move.from_uci('b8c6'))
testBoard.push(chess.Move.from_uci('h5f7'))
print(testBoard)

print(features(testBoard))

r . b q k b . r
p p p p . Q p p
. . n . . n . .
. . . . p . . .
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R
[1, 0, 0, 0, 8, 2, 2, 2, 1, 1, 7, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [43]:
def length_of_game(game):
    return sum(1 for e in game.mainline_moves())

def result(game):
    result = game.headers["Result"]
    answer = 0
    if result == '1-0':
        answer = 1
    elif result == '0-1':
        answer = -1
    return answer

In [44]:
def add_game_to_dataset(array, game):
    board = game.board()
    length_game = length_of_game(game)
    current_move = 0
    result = game.headers["Result"]
    multiplier = 0
    if result == '1-0':
        multiplier = 1
    elif result == '0-1':
        multiplier = -1
    for move in game.mainline_moves():
        current_move += 1
        board.push(move)
        current_features = features(board)
        current_features.append(multiplier * current_move / length_game)
        array.append(current_features)

In [59]:
def games_to_dataset(games):
    data_dict = dict()
    def add_to_datadict(features, result):
        tupfeatures = tuple(features)
        if(data_dict.get(tupfeatures) is not None):
            data = data_dict.get(tupfeatures)
        else:
            data = (0,0)
        data = list(data)
        data[0] = data[0] + result
        data[1] = data[1] + 1
        data = tuple(data)
        data_dict[tupfeatures] = data
    for game in games:
        board = game.board()
        pts = result(game)
        for move in game.mainline_moves():
            board.push(move)
            current_features = features(board)
            add_to_datadict(current_features, pts)
    dataset = []
    for key in data_dict:
        cur_row = list(key)
        cur_row.append(data_dict[key][0] / data_dict[key][1])
        dataset.append(cur_row)
    return dataset
        

            

In [96]:
#IMPORT THE DATASET

modelgameslist = gameslist[:300]
dataset = games_to_dataset(modelgameslist)
print(len(dataset))
print(sum([1 for i in dataset if i[28] > 0]))
data = pd.DataFrame(dataset, columns=columns)
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
data

9899
4721


,wMate,bMate,draw,turn,wP,wN,wB,wR,wQ,wK,...,hwR,hwQ,hwK,hbP,hbN,hbB,hbR,hbQ,hbK,EVAL
0,0,0,0,0,8,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0.105647
1,0,0,0,1,8,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,-0.018519
2,0,0,0,0,8,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0.044444
3,0,0,0,1,7,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0.086432
4,0,0,0,0,7,2,2,2,1,1,...,0,0,0,0,0,0,0,1,0,-0.122222
5,0,0,0,0,7,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0.139198
6,0,0,0,0,7,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,-0.279070
7,0,0,0,1,6,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0.007752
8,0,0,0,0,6,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,1.000000
9,0,0,0,1,5,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,1.000000


In [97]:
#SPLIT TRAIN AND TEST SET
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [98]:
#FEATURE SCALING
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train = mms.fit_transform(x_train)
x_test = mms.transform(x_test)
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [112]:
#BUILDING THE ANN
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=28, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [113]:
#TRAINING THE ANN
ann.compile(optimizer='adam', loss='mean_squared_error')
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
248/248 [==============================] - 0s 804us/step - loss: 0.8527
Epoch 2/100
248/248 [==============================] - 0s 776us/step - loss: 0.7744
Epoch 3/100
248/248 [==============================] - 0s 843us/step - loss: 0.7319
Epoch 4/100
248/248 [==============================] - 0s 864us/step - loss: 0.7023
Epoch 5/100
248/248 [==============================] - 0s 832us/step - loss: 0.6862
Epoch 6/100
248/248 [==============================] - 0s 816us/step - loss: 0.6736
Epoch 7/100
248/248 [==============================] - 0s 796us/step - loss: 0.6652
Epoch 8/100
248/248 [==============================] - 0s 857us/step - loss: 0.6594
Epoch 9/100
248/248 [==============================] - 0s 864us/step - loss: 0.6555
Epoch 10/100
248/248 [==============================] - 0s 788us/step - loss: 0.6497
Epoch 11/100
248/248 [==============================] - 0s 869us/step - loss: 0.6473
Epoch 12/100
248/248 [==============================] - 0s 800us/step - lo

248/248 [==============================] - 0s 816us/step - loss: 0.5573
Epoch 98/100
248/248 [==============================] - 0s 812us/step - loss: 0.5578
Epoch 99/100
248/248 [==============================] - 0s 800us/step - loss: 0.5567
Epoch 100/100
248/248 [==============================] - 0s 853us/step - loss: 0.5559


In [114]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
#testBoard.push(chess.Move.from_uci('d1h5'))
#testBoard.push(chess.Move.from_uci('g8f6'))
#testBoard.push(chess.Move.from_uci('f1c4'))
#testBoard.push(chess.Move.from_uci('f6h5'))
testarray = [features(testBoard)]
mms.transform(testarray)
print(testarray)
ann.predict(testarray)


[[0, 0, 0, 1, 8, 2, 2, 2, 1, 1, 8, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


array([[-6.457924]], dtype=float32)